# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data:

In [ ]:
#Dependencies and Setup:
import hvplot.pandas
import pandas as pd
import requests

#Import API key:
from api_keys import geoapify_key

In [ ]:
#Load the CSV file created in Part 1 into a Pandas DataFrame:
city_data_df = pd.read_csv("Output_Data/cities.csv")

#Display sample data:
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hawaiian paradise park,19.5933,-154.9731,300.88,82,100,4.12,US,1690159199
1,1,pisco,-13.7000,-76.2167,294.18,68,100,5.14,PE,1690159204
2,2,puerto natales,-51.7236,-72.4875,268.94,92,15,1.49,CL,1690159205
3,3,invercargill,-46.4000,168.3500,282.98,88,96,0.11,NZ,1690159205
4,4,sandnessjoen,66.0217,12.6316,284.44,95,100,1.40,NO,1690159205


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city:

In [ ]:
%%capture --no-display

#Configure the map plot:
map_plot_city = city_data_df.hvplot.points("Lng","Lat", geo = True, tiles="OSM", \
                                        frame_width=700, frame_height=500, \
                                        size="Humidity", scale=0.5, color="City")

#Display the map:
map_plot_city

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition:

In [ ]:
#Narrow down cities that fit criteria and drop any results with null values:
new_data_df = pd.DataFrame(city_data_df, columns = ["City", "Max Temp", "Wind Speed", "Cloudiness"])

max_temp = (new_data_df["Max Temp"] <= 27) & (new_data_df["Max Temp"] > 21)
wind_speed = (new_data_df["Wind Speed"] < 10)
cloudiness = (new_data_df["Cloudiness"] == 0)

#Drop any rows with null values:
new_data_df[max_temp & wind_speed & cloudiness]

new_data_df = (city_data_df.dropna())

#Display sample data:
new_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hawaiian paradise park,19.5933,-154.9731,300.88,82,100,4.12,US,1690159199
1,1,pisco,-13.7000,-76.2167,294.18,68,100,5.14,PE,1690159204
2,2,puerto natales,-51.7236,-72.4875,268.94,92,15,1.49,CL,1690159205
3,3,invercargill,-46.4000,168.3500,282.98,88,96,0.11,NZ,1690159205
4,4,sandnessjoen,66.0217,12.6316,284.44,95,100,1.40,NO,1690159205
...,...,...,...,...,...,...,...,...,...,...
560,560,whittlesea,-32.1759,26.8244,278.38,87,54,2.53,ZA,1690159356
561,561,ponnani,10.7670,75.9252,295.77,89,100,2.49,IN,1690159357
562,562,margate,51.3813,1.3862,288.64,89,78,2.24,GB,1690159357
563,563,anyama,5.4946,-4.0518,297.60,87,100,1.58,CI,1690159357


### Step 3: Create a new DataFrame called `hotel_df`.

In [ ]:
#Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity:
hotel_df = pd.DataFrame(city_data_df, columns=["City", "Country", "Lat", "Lng", "Humidity"])

#Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API:
hotel_df["Hotel Name"] = ""

#Display sample data:
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,hawaiian paradise park,US,19.5933,-154.9731,82,
1,pisco,PE,-13.7000,-76.2167,68,
2,puerto natales,CL,-51.7236,-72.4875,92,
3,invercargill,NZ,-46.4000,168.3500,88,
4,sandnessjoen,NO,66.0217,12.6316,95,
...,...,...,...,...,...,...
560,whittlesea,ZA,-32.1759,26.8244,87,
561,ponnani,IN,10.7670,75.9252,89,
562,margate,GB,51.3813,1.3862,89,
563,anyama,CI,5.4946,-4.0518,87,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [ ]:
#Set parameters to search for a hotel:
radius = 10000
limit = 5
params = {
    "limit":limit,
    "apiKey":geoapify_key 
}

#Print a message to follow up the hotel search:
print("Starting hotel search")

#Iterate through the hotel_df DataFrame:
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame:
    longitude = hotel_df.loc[index, "Lng"]
    latitude = hotel_df.loc[index, "Lat"]
    
    
    #Add filter and bias parameters with the current city's latitude and longitude to the params dictionary:
    params["filter"] = f'circle:{longitude},{latitude},{radius}'
    params["bias"] = f'proximity:{longitude},{latitude}'
    #params["categories"] = f'accommodation.hotel'

    #Get base URL:
    base_url = "https://api.geoapify.com/v2/places"


    #Make an API request using the params dictionary:
    name_address = requests.get(base_url, params=params)
    
    #Convert the API response to a JSON format:
    name_address = name_address.json()
    
    #Grab the first hotel from the results and store the name in the hotel_df DataFrame:
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    #Log the search results:
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

#Display sample data:
hotel_df

Starting hotel search
hawaiian paradise park - nearest hotel: No hotel found
pisco - nearest hotel: No hotel found
puerto natales - nearest hotel: No hotel found
invercargill - nearest hotel: No hotel found
sandnessjoen - nearest hotel: No hotel found
albany - nearest hotel: No hotel found
mount pearl - nearest hotel: No hotel found
patong - nearest hotel: No hotel found
pamandzi - nearest hotel: No hotel found
souillac - nearest hotel: No hotel found
blackmans bay - nearest hotel: No hotel found
narrabri - nearest hotel: No hotel found
puerto ayora - nearest hotel: No hotel found
whitehorse - nearest hotel: No hotel found
bredasdorp - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
nyeri - nearest hotel: No hotel found
iskateley - nearest hotel: No hotel found
tungao - nearest hotel: No hotel found
port alfred - nearest hotel: No hotel found
rongelap - nearest hotel: No hotel found
hithadhoo - nearest hotel: No hotel found
lorengau - nearest hotel: No hotel fou

,City,Country,Lat,Lng,Humidity,Hotel Name
0,hawaiian paradise park,US,19.5933,-154.9731,82,No hotel found
1,pisco,PE,-13.7000,-76.2167,68,No hotel found
2,puerto natales,CL,-51.7236,-72.4875,92,No hotel found
3,invercargill,NZ,-46.4000,168.3500,88,No hotel found
4,sandnessjoen,NO,66.0217,12.6316,95,No hotel found
...,...,...,...,...,...,...
560,whittlesea,ZA,-32.1759,26.8244,87,No hotel found
561,ponnani,IN,10.7670,75.9252,89,No hotel found
562,margate,GB,51.3813,1.3862,89,No hotel found
563,anyama,CI,5.4946,-4.0518,87,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

#Configure the map plot:
map_plot_hotel = hotel_df.hvplot.points("Lng","Lat", geo = True, tiles="OSM", \
                                        frame_width=700, frame_height=500, \
                                        size="Humidity", scale=0.5, color="City", \
                                        hover_cols = ["Hotel Name","Country"]
                                       )

#Display the map:
map_plot_hotel

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)